## Use cBioPortal APIs to reproduce survival chart BRAF V600E mut vs wildtype

In [1]:
pip install bravado

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 130 kB 7.7 MB/s 
     |████████████████████████████████| 67 kB 6.6 MB/s 
     |████████████████████████████████| 66 kB 4.7 MB/s 


In [2]:
!git clone https://github.com/cbioportal/workbench

fatal: destination path 'workbench' already exists and is not an empty directory.


In [3]:
%config Completer.use_jedi = False

## Following this tutorial
https://github.com/cBioPortal/codebook/blob/main/python/recipes/survival-charts.ipynb

### Errors

* 302 error -> old url
    * fix in: 
https://groups.google.com/g/cbioportal/c/eadClFL6V6w?pli=1
    ```
    Bad: https://www.cbioportal.org/api/api-docs
    Good: https://www.cbioportal.org/api/v2/api-docs

    ```

* bad swagger expecification
    * fix in https://groups.google.com/g/cbioportal/c/PTVmjR6HTjg
    ```
    Bad:
     config={"validate_requests":False,"validate_responses":False})

    Good:
    config={"validate_requests":False,"validate_responses":False,"validate_swagger_spec": False,})
    ```

In [4]:
from bravado.client import SwaggerClient

cbioportal = SwaggerClient.from_url('https://www.cbioportal.org/api/v2/api-docs',
                                    config={"validate_requests":False,"validate_responses":False,"validate_swagger_spec": False,})
print(cbioportal)

SwaggerClient(https://www.cbioportal.org/api)


In [5]:
study_id='skcm_tcga_pan_can_atlas_2018'

In [6]:
import pandas as pd

def get_pandas_df(list):
    return pd.DataFrame.from_dict([
            dict(
                {k:getattr(m,k) for k in m})
                    for m in list
                    ])

In [7]:
mol_profiles=cbioportal.Molecular_Profiles.getAllMolecularProfilesInStudyUsingGET(studyId=study_id).result()
mpdf = get_pandas_df(mol_profiles)
mpdf.head()

,datatype,description,genericAssayType,molecularAlterationType,molecularProfileId,name,patientLevel,pivotThreshold,showProfileInAnalysisTab,sortOrder,study,studyId
0,LOG2-VALUE,Protein expression measured by reverse-phase p...,None,PROTEIN_LEVEL,skcm_tcga_pan_can_atlas_2018_rppa,Protein expression (RPPA),False,NaN,False,None,None,skcm_tcga_pan_can_atlas_2018
1,Z-SCORE,"Protein expression, measured by reverse-phase ...",None,PROTEIN_LEVEL,skcm_tcga_pan_can_atlas_2018_rppa_Zscores,Protein expression z-scores (RPPA),False,NaN,True,None,None,skcm_tcga_pan_can_atlas_2018
2,DISCRETE,Putative copy-number from GISTIC 2.0. Values: ...,None,COPY_NUMBER_ALTERATION,skcm_tcga_pan_can_atlas_2018_gistic,Putative copy-number alterations from GISTIC,False,NaN,True,None,None,skcm_tcga_pan_can_atlas_2018
3,LOG2-VALUE,Log2 copy-number values for each gene (from Af...,None,COPY_NUMBER_ALTERATION,skcm_tcga_pan_can_atlas_2018_log2CNA,Log2 copy-number values,False,NaN,False,None,None,skcm_tcga_pan_can_atlas_2018
4,CATEGORICAL,Putative arm-level copy-number from GISTIC 2.0.,ARMLEVEL_CNA,GENERIC_ASSAY,skcm_tcga_pan_can_atlas_2018_armlevel_cna,Putative arm-level copy-number from GISTIC,False,NaN,True,None,None,skcm_tcga_pan_can_atlas_2018


In [8]:
mol_profile_id = mpdf.loc[mpdf['name'] == 'Mutations']['molecularProfileId'].item()
mol_profile_id

'skcm_tcga_pan_can_atlas_2018_mutations'

In [9]:
genes = cbioportal.Genes.getAllGenesUsingGET(keyword='BRAF').result()
gdf = get_pandas_df(genes)
gdf.head()

,entrezGeneId,geneticEntityId,hugoGeneSymbol,type
0,286494,None,BRAFP1,pseudogene
1,673,None,BRAF,protein-coding
2,-3990,None,BRAF_PS445,phosphoprotein


In [10]:
entrez_gene_id = gdf.loc[gdf['hugoGeneSymbol'] == 'BRAF']['entrezGeneId'].item()
entrez_gene_id

673

## Stratify altered / unaltered patients

In [11]:
mutations = cbioportal.Mutations.getMutationsInMolecularProfileBySampleListIdUsingGET(
    sampleListId='skcm_tcga_pan_can_atlas_2018_cnaseq', 
    molecularProfileId=mol_profile_id,
    entrezGeneId=entrez_gene_id).result()

mdf = get_pandas_df(mutations)
mdf.columns

Index(['alleleSpecificCopyNumber', 'aminoAcidChange', 'center', 'chr',
       'driverFilter', 'driverFilterAnnotation', 'driverTiersFilter',
       'driverTiersFilterAnnotation', 'endPosition', 'entrezGeneId',
       'fisValue', 'functionalImpactScore', 'gene', 'keyword', 'linkMsa',
       'linkPdb', 'linkXvar', 'molecularProfileId', 'mutationStatus',
       'mutationType', 'namespaceColumns', 'ncbiBuild', 'normalAltCount',
       'normalRefCount', 'patientId', 'proteinChange', 'proteinPosEnd',
       'proteinPosStart', 'referenceAllele', 'refseqMrnaId', 'sampleId',
       'startPosition', 'studyId', 'tumorAltCount', 'tumorRefCount',
       'uniquePatientKey', 'uniqueSampleKey', 'validationStatus',
       'variantAllele', 'variantType'],
      dtype='object')

In [12]:
all_patients = mdf['patientId'].tolist()
len(set(all_patients))

188

In [13]:
altered_mdf = mdf[mdf['proteinChange']=='V600E']
altered_mdf.head()

,alleleSpecificCopyNumber,aminoAcidChange,center,chr,driverFilter,driverFilterAnnotation,driverTiersFilter,driverTiersFilterAnnotation,endPosition,entrezGeneId,...,sampleId,startPosition,studyId,tumorAltCount,tumorRefCount,uniquePatientKey,uniqueSampleKey,validationStatus,variantAllele,variantType
21,None,None,.,7,None,None,None,None,140453136,673,...,TCGA-D3-A1Q4-06,140453136,skcm_tcga_pan_can_atlas_2018,26,29,VENHQS1EMy1BMVE0OnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,VENHQS1EMy1BMVE0LTA2OnNrY21fdGNnYV9wYW5fY2FuX2...,.,T,SNP
22,None,None,.,7,None,None,None,None,140453136,673,...,TCGA-D3-A1Q5-06,140453136,skcm_tcga_pan_can_atlas_2018,33,17,VENHQS1EMy1BMVE1OnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,VENHQS1EMy1BMVE1LTA2OnNrY21fdGNnYV9wYW5fY2FuX2...,.,T,SNP
23,None,None,.,7,None,None,None,None,140453136,673,...,TCGA-D3-A1Q6-06,140453136,skcm_tcga_pan_can_atlas_2018,17,36,VENHQS1EMy1BMVE2OnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,VENHQS1EMy1BMVE2LTA2OnNrY21fdGNnYV9wYW5fY2FuX2...,.,T,SNP
24,None,None,.,7,None,None,None,None,140453136,673,...,TCGA-D3-A1Q7-06,140453136,skcm_tcga_pan_can_atlas_2018,4,33,VENHQS1EMy1BMVE3OnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,VENHQS1EMy1BMVE3LTA2OnNrY21fdGNnYV9wYW5fY2FuX2...,.,T,SNP
25,None,None,.,7,None,None,None,None,140453136,673,...,TCGA-D3-A1Q9-06,140453136,skcm_tcga_pan_can_atlas_2018,31,47,VENHQS1EMy1BMVE5OnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,VENHQS1EMy1BMVE5LTA2OnNrY21fdGNnYV9wYW5fY2FuX2...,.,T,SNP


In [14]:
altered_patients = altered_mdf['patientId'].tolist()
len(set(altered_patients))

128

In [15]:
mdf["aminoAcidChange"].unique().tolist()

[None]

In [16]:
mdf["chr"].unique().tolist()

['7']

In [17]:
unaltered_patients = list(set(all_patients) - set(altered_patients))
len(unaltered_patients)

60

In [18]:
attribute_ids = ["OS_STATUS","OS_MONTHS","DFS_STATUS","DFS_MONTHS","DSS_STATUS","DSS_MONTHS","PFS_STATUS","PFS_MONTHS"]

clinical_data_single_study_filter = {
  'attributeIds': attribute_ids,
  'ids': altered_patients
}

altered_clinical = cbioportal.Clinical_Data.fetchAllClinicalDataInStudyUsingPOST(studyId = study_id, 
                                                              clinicalDataType='PATIENT',
                                                              clinicalDataSingleStudyFilter=clinical_data_single_study_filter).result()
altered_clinical_df = get_pandas_df(altered_clinical)
altered_clinical_df.head()

,clinicalAttribute,clinicalAttributeId,patientId,sampleId,studyId,uniquePatientKey,uniqueSampleKey,value
0,None,DSS_MONTHS,TCGA-D3-A1Q4,None,skcm_tcga_pan_can_atlas_2018,VENHQS1EMy1BMVE0OnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,112.0426078
1,None,DSS_STATUS,TCGA-D3-A1Q4,None,skcm_tcga_pan_can_atlas_2018,VENHQS1EMy1BMVE0OnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,0:ALIVE OR DEAD TUMOR FREE
2,None,OS_MONTHS,TCGA-D3-A1Q4,None,skcm_tcga_pan_can_atlas_2018,VENHQS1EMy1BMVE0OnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,112.0426078
3,None,OS_STATUS,TCGA-D3-A1Q4,None,skcm_tcga_pan_can_atlas_2018,VENHQS1EMy1BMVE0OnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,0:LIVING
4,None,PFS_MONTHS,TCGA-D3-A1Q4,None,skcm_tcga_pan_can_atlas_2018,VENHQS1EMy1BMVE0OnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,112.0426078


In [19]:
#altered_clinical_df
altered_os = altered_clinical_df.loc[(altered_clinical_df.clinicalAttributeId == "OS_MONTHS") | (altered_clinical_df.clinicalAttributeId == "OS_STATUS")]
altered_os

,clinicalAttribute,clinicalAttributeId,patientId,sampleId,studyId,uniquePatientKey,uniqueSampleKey,value
2,None,OS_MONTHS,TCGA-D3-A1Q4,None,skcm_tcga_pan_can_atlas_2018,VENHQS1EMy1BMVE0OnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,112.0426078
3,None,OS_STATUS,TCGA-D3-A1Q4,None,skcm_tcga_pan_can_atlas_2018,VENHQS1EMy1BMVE0OnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,0:LIVING
7,None,OS_MONTHS,TCGA-D3-A1Q5,None,skcm_tcga_pan_can_atlas_2018,VENHQS1EMy1BMVE1OnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,112.5686294
8,None,OS_STATUS,TCGA-D3-A1Q5,None,skcm_tcga_pan_can_atlas_2018,VENHQS1EMy1BMVE1OnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,1:DECEASED
13,None,OS_MONTHS,TCGA-D3-A1Q6,None,skcm_tcga_pan_can_atlas_2018,VENHQS1EMy1BMVE2OnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,71.80195286
...,...,...,...,...,...,...,...,...
736,None,OS_MONTHS,TCGA-WE-AAA4,None,skcm_tcga_pan_can_atlas_2018,VENHQS1XRS1BQUE0OnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,24.98602755
737,None,OS_STATUS,TCGA-WE-AAA4,None,skcm_tcga_pan_can_atlas_2018,VENHQS1XRS1BQUE0OnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,0:LIVING
741,None,OS_STATUS,TCGA-YD-A9TB,None,skcm_tcga_pan_can_atlas_2018,VENHQS1ZRC1BOVRCOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,0:LIVING
745,None,OS_MONTHS,TCGA-Z2-AA3V,None,skcm_tcga_pan_can_atlas_2018,VENHQS1aMi1BQTNWOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,15.97790709


In [20]:
clinical_data_single_study_filter = {
  'attributeIds': attribute_ids,
  'ids': unaltered_patients
}

unaltered_clinical = cbioportal.Clinical_Data.fetchAllClinicalDataInStudyUsingPOST(studyId = study_id, 
                                                              clinicalDataType='PATIENT',
                                                              clinicalDataSingleStudyFilter=clinical_data_single_study_filter).result()
unaltered_clinical_df = get_pandas_df(unaltered_clinical)
unaltered_clinical_df.head()

,clinicalAttribute,clinicalAttributeId,patientId,sampleId,studyId,uniquePatientKey,uniqueSampleKey,value
0,None,DSS_MONTHS,TCGA-3N-A9WB,None,skcm_tcga_pan_can_atlas_2018,VENHQS0zTi1BOVdCOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,17.02995036
1,None,DSS_STATUS,TCGA-3N-A9WB,None,skcm_tcga_pan_can_atlas_2018,VENHQS0zTi1BOVdCOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,1:DEAD WITH TUMOR
2,None,OS_MONTHS,TCGA-3N-A9WB,None,skcm_tcga_pan_can_atlas_2018,VENHQS0zTi1BOVdCOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,17.02995036
3,None,OS_STATUS,TCGA-3N-A9WB,None,skcm_tcga_pan_can_atlas_2018,VENHQS0zTi1BOVdCOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,1:DECEASED
4,None,PFS_MONTHS,TCGA-3N-A9WB,None,skcm_tcga_pan_can_atlas_2018,VENHQS0zTi1BOVdCOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,14.00532597


## Overal Survival analysis


In [21]:
#altered_clinical_df
altered_os = altered_clinical_df.loc[(altered_clinical_df.clinicalAttributeId == "OS_MONTHS") | (altered_clinical_df.clinicalAttributeId == "OS_STATUS")]
print(len(altered_os))
altered_dss = altered_clinical_df.loc[(altered_clinical_df.clinicalAttributeId == "DSS_MONTHS") | (altered_clinical_df.clinicalAttributeId == "DSS_STATUS")]
print(len(altered_dss))
altered_pfs = altered_clinical_df.loc[(altered_clinical_df.clinicalAttributeId == "PFS_MONTHS") | (altered_clinical_df.clinicalAttributeId == "PFS_STATUS")]
print(len(altered_pfs))

250
248
251


In [22]:
#unaltered_clinical_df
unaltered_os = unaltered_clinical_df.loc[(unaltered_clinical_df.clinicalAttributeId == "OS_MONTHS") | (unaltered_clinical_df.clinicalAttributeId == "OS_STATUS")]
print(len(unaltered_os))
unaltered_dss = unaltered_clinical_df.loc[(unaltered_clinical_df.clinicalAttributeId == "DSS_MONTHS") | (unaltered_clinical_df.clinicalAttributeId == "DSS_STATUS")]
print(len(unaltered_dss))
unaltered_pfs = unaltered_clinical_df.loc[(unaltered_clinical_df.clinicalAttributeId == "PFS_MONTHS") | (unaltered_clinical_df.clinicalAttributeId == "PFS_STATUS")]
print(len(unaltered_pfs))

114
114
114


## Prep survival data

In [23]:
def prep_months_status(df):
    months = df.loc[df.clinicalAttributeId.str.contains("MONTH")][['patientId', 'value']]
    status = df.loc[df.clinicalAttributeId.str.contains("STATUS")][['patientId', 'value']]
    # 0:LIVING, 1:DECEASED
    status['value'] = status['value'].apply(lambda x: x.split(":")[0])

    # merge months/status and sort by numeric month values
    return months.merge(status, on='patientId', suffixes=['_month','_status']) \
                .astype({'value_status': 'float32', 'value_month': 'float32'}) \
                .sort_values('value_month', ascending=False)


uos = prep_months_status(unaltered_os)
udss = prep_months_status(unaltered_dss)
upfs = prep_months_status(unaltered_pfs)
aos = prep_months_status(altered_os)
adss = prep_months_status(altered_dss)
apfs = prep_months_status(altered_pfs)


## Get KaplanMeier cumulative density data

In [24]:
pip install lifelines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
from lifelines import KaplanMeierFitter

def get_km_cd(df):
    kmf = KaplanMeierFitter() 
    kmf.fit(df['value_month'], df['value_status'], label='Kaplan Meier Estimate')

    # get cdf from KMF
    return kmf.cumulative_density_.reset_index()

uos_km = get_km_cd(uos)
udss_km = get_km_cd(udss)
upfs_km = get_km_cd(upfs)
aos_km = get_km_cd(aos)
adss_km = get_km_cd(adss)
apfs_km = get_km_cd(apfs)

## Kaplan-Meier curves with Plotly

In [26]:
pip install plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=aos.value_month,
    y=aos_km['Kaplan Meier Estimate'],
    name='altered',
    mode='lines+markers',
    text=aos.patientId
))

fig.add_trace(go.Scatter(
    x=uos.value_month,
    y=uos_km['Kaplan Meier Estimate'],
    name='unaltered',
    mode='lines+markers',
    text=uos.patientId
))

fig.update_layout(title_text="Overall Survival")
fig.show()

In [28]:
pip install jupyter-dash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
from jupyter_dash import JupyterDash

import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd


external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.RadioItems(
                id='survival-type',
                options=[{'label': i, 'value': i} for i in ['Overall', 'Disease Specific', 'Progression Free']],
                value='Overall',
                labelStyle={'display': 'inline-block'}
            )
        ], style={'display': 'inline-block'})
    ], style={
        'borderBottom': 'thin lightgrey solid',
        'backgroundColor': 'rgb(250, 250, 250)',
        'padding': '10px 5px'
    }),

    html.Div([
        dcc.Graph(
            id='survival-scatter'
        )
    ], style={'display': 'inline-block', 'padding': '0 20'}),

    html.Div(dcc.Slider(
        id='month-slider',
        min=0, #aos['value_month'].min(),
        max=370, #aos['value_month'].max(),
        value=aos['value_month'].max(),
        marks={str(int(month)): str(int(month)) for month in aos['value_month'].unique()},
        step=None
    ), style={'padding': '0px 20px 20px 20px'})
])

@app.callback(
    dash.dependencies.Output('survival-scatter', 'figure'),
    [dash.dependencies.Input('month-slider', 'value'),
     dash.dependencies.Input('survival-type', 'value')])
def update_graph(max_month, survival_type):
    # warning, hacky code
    if survival_type == 'Overall':
        adf = aos[aos['value_month'] <= max_month]
        udf = uos[uos['value_month'] <= max_month]
        akm = aos_km
        ukm = uos_km
        
    if survival_type == 'Disease Specific':
        adf = adss[adss['value_month'] <= max_month]
        udf = udss[udss['value_month'] <= max_month]
        akm = adss_km
        ukm = udss_km
        
    if survival_type == 'Progression Free':
        adf = apfs[apfs['value_month'] <= max_month]
        udf = upfs[upfs['value_month'] <= max_month]
        akm = apfs_km
        ukm = upfs_km
        
    return {
        'data': [dict(
            x=adf['value_month'],
            y=akm['Kaplan Meier Estimate'],
            name='altered',
            text=aos['patientId'],
            mode='lines+markers',
            ),
            dict(
            x=udf['value_month'],
            y=ukm['Kaplan Meier Estimate'],
            name='unaltered',
            text=aos['patientId'],
            mode='lines+markers',
        )],
        'layout': dict(
            xaxis={
                'title': 'months',
            },
            yaxis={
                'title': 'survival rate',
            },
            margin={'l': 40, 'b': 30, 't': 10, 'r': 0},
            height=450,
            hovermode='closest'
        )
    }

app.run_server(mode="inline")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning:


The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning:


The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`



<IPython.core.display.Javascript object>

In [30]:
# app.run_server()